### Extraktion der Texte vom Gesamtkorpus des Deutschen Textarchivs und Einspeisung in SQLite-Datenbank ###

Dieses Jupyter-Notebook dient der Nachprüfbarkeit der Schritte für die Extraktion der Texte und Metadaten von der vom DTA zur Verfügung gestellten XML-Dateien, und die Einspeisung dieser Daten in eine SQLite-Datenbank. Dies ermöglicht eine erleichterte Handhabung, bei der Datenanalyse.

## Requirements ##
- Pandas für die to_sql()-function, die von einem Pandas DataFrame eine SQL-Datenbank erstellt (Pandas wird später noch mehr gebraucht)
- os, für das rekursive Iterieren des Directory's mit den XML-Dateien
- sqlite3, für das Kreieren von der Datenbank
- xml.etree.Elementree, für die Extraktion von Texten und Metadaten von den XML-Dateien
- zipfile, für das Entzippen des Textkorpus

## 0. Auswählen eines Python Kernels / Setup eines Virtual environments (good practice) ##
Zunächst muss ein Kernel ausgewählt werden, damit Jupyter-Notebook Python code ausführen kann.

Um die Installation von libraries zu vereinfachen, und die Arbeitsumgebung sauber zu halten, wird ein virtual environment (venv) erstellt. Darin werden dann alle libraries und packages lokal installiert, anstatt global auf einer Python Machine. 

In [1]:
!python -m venv HistTopMod
#Wenn ein Popup-Fenster fragt, ob dieses Environment benutzt werden soll: Mit Ja antworten.

Als nächstes muss in diesem Virtual Environment ipykernel installiert werden, damit das venv als Kernel verwendet werden kann.

In [2]:
!HistTopMod\Scripts\python -m pip install notebook ipykernel

  Using cached notebook-7.3.2-py3-none-any.whl.metadata (10 kB)
  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jupyter_server-2.15.0-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyterlab_server-2.27.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached jupyterlab-4.3.4-py3-none-any.whl.metadata (16 kB)
  Using cached notebook_shim-0.2.4-py3-none-any.whl.metadata (4.0 kB)
  Using cached tornado-6.4.2-cp38-abi3-win_amd64.whl.metadata (2.6 kB)
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached debugpy-1.8.11-cp312-cp312-win_amd64.whl.metadata (1.1 kB)
  Using cached ipython-8.31.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached jupyter_client-8.6.3-py3-none-any.whl.metadata (8.3 kB)
  Using cached jupyter_core-5.7.2-py3-none-any.whl.metadata (3.4 kB)
  Using cached matplotlib_inline-0.1.7-py3-none-any.whl.metadata (3.9 kB)
  Using cached nest_asyncio-1.6.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached packaging-24.2-py


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: c:\Users\raoul\Desktop\new_histtop\HistTopMod-Essay\HistTopMod\Scripts\python.exe -m pip install --upgrade pip


Nun kann man das erstellte venv als Kernel auswählen (Name: "HistTopMod")

## 1. Installieren der notwendigen Requirements ##
Zunächst müssen alle Libraries importiert werden. Während os, sqlite3 und xml.etree.Elementree in Python built-in sind, muss pandas mit pip installiert werden.

In [1]:
import os
import sqlite3
import xml.etree.ElementTree as ET
import zipfile

Pandas muss zunächst mit pip installiert werden:

In [2]:
%pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 330.3 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 361.5 kB/s eta 0:00:00
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.6 MB 3.0 MB/s eta 0:00:05
   - -------------------------------------- 0.3/12.6 MB 3.9 MB/s eta 0:00:04
   -- ------------------------------------- 0.8/12.6 MB 6.0 MB/s eta 0:00:02
   -- ------------------------------------- 0.9/


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Danach kann die library auch importiert werden:

In [3]:
import pandas as pd

## 2. Herunterladen vom Gesamtkorpus des Deutschen Textarchivs ##
Als nächstes wird der Gesamtkorpus des Deutschen Textarchivs in das working directory heruntergeladen und der Path gespeichert.

Der Download des Gesamtkorpus kann unter dieser URL gestarted werden:

https://www.deutschestextarchiv.de/media/download/dta_komplett_2021-05-13.zip

Das Zip-File wird nun in dieses Working Directory kopiert. Es muss sichergestellt werden, dass es sich im selben Ordner befindet.

## 3. Entpacken der Zip-Datei und speichern des Path ##
Die folgenden Code-Snippets kreieren ein XML-Directory mit allen XML-Files des DTA, entpacken die Zip-Datei in dieses Directory und löschen den Zip-Ordner.

Zunächst wird ein neuer Ordner erstellt und der Path zu diesem Ordner gespeichert:

In [ ]:
!mkdir "Gesamtkorpus"
path_to_directory = r"./Gesamtkorpus"

Als nächstes werden die Inhalte des Zip-Files "dta_komplett_2021-05-13.zip" in den Ordner "Gesamtkorpus" entpackt.

In [ ]:
path_to_zip_file = "./dta_komplett_2021-05-13.zip"
with zipfile.ZipFile(path_to_zip_file, "r") as ref:
    ref.extractall(path_to_directory)

Damit nun auf die Dateien zugegriffen werden kann, muss path_to_directory upgedated werden (gibt eine schönere Lösung)

In [14]:
path_to_directory = path_to_directory + "/" + "dta_komplett_2021-05-13/"

Schliesslich wird das Zip-File gelöscht, da es nicht mehr benötigt wird.

In [14]:
os.remove("dta_komplett_2021-05-13.zip")

## 4. Erstellen einer SQL-Datenbank und verbinden mit ihr ##
In diesem Schritt wird eine SQL-Datenbank erstellt und eine Verbindung aufgebaut, dies wird später gebraucht, um die extrahierten Texte und Metadaten in die Datenbank zu kopieren.

In [28]:
con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

## 5. Extrahieren von Metadaten und Texte ##
In diesem Schritt werden die einzelnen Metadaten und Texte von den XML-Dateien extrahiert und als Dictionaries in eine Liste kopiert. Diese Liste wird dann in ein pandas DataFrame umgewandelt und in die SQL-Datenbank von Schritt 4 eingefügt.

Es folgt nun eine Python-Function, die rekursiv Text-Elemente extrahiert. Die Funktion ist relativ kompliziert (Rekursion), aber gewährleistet, dass XML-Tags nicht im Text übernommen werden.
Die XML-Textteile weisen oft noch Formatierungen mit solchen Tags auf (z.B. ```<lb>``` für linebreaks). Bei der Extraktion sollen diese aber nicht vorkommen: Das Ziel ist es, einen Reintext zu haben ohne Formatierungen.
Beispiel:

```XML
<text>
        <body>
            <pb facs="#f0001" n="98" />
            <div n="1">
                <head>
                    <hi rendition="#b"><hi rendition="#g">Geschichte eines deutschen Liedes</hi>.</hi>
                    <lb />
                </head>
                <cit>
                    <quote>
                        <lg type="poem">
                            <l>„Ein Veilchen auf der Wiese stand,<lb />
                            </l>
                            <l>Gebückt in sich und unbekannt:<lb />
                            </l>
                            <l>Es war ein herzigs Veilchen.“<lb />
                            </l>
                        </lg>
                    </quote>
                    <bibl>
                        <hi rendition="#right">(Goethe – Mozart)</hi>
                    </bibl>
                    .........
```

Mit dem folgenden Code können alle subtags herausgelöscht werden, sodass schliesslich der Output so aussieht:
```
Geschichte eines deutschen Liedes
„Ein Veilchen auf der Wiese stand,
Gebückt in sich und unbekannt:
Es war ein herzigs Veilchen.“
(Goethe – Mozart)
```

Der Code speichert diese Funktion zunächst nur, führt sie aber noch nicht aus! Sie wird beim nächsten Code-Snippet verwendet.


In [4]:
def extract_text(text_element):
    # Hier wird der extrahierte Text hinein kopiert
    text_content = []

    # Helfer-Funktion um rekursiv Text vom text_element und seinen Children zu extrahieren
    def recursive_extract(text_elem):
        # Wenn das aktuelle element Text beinhaltet, wird dieser in text_content reinkopiert
        if text_elem.text:
            text_content.append(text_elem.text)
        
        # Iteration über alle Child-Elemente des aktuellen Elements
        for child in text_elem:
            # Rekursiver Function-Call für jedes Child-Element
            recursive_extract(child)
            
            # zur Sicherheit: Falls Text hinter closing Tags sind (z.B. ...Wiese stand, <lb />AUSSERHALB) wird dieser auch in text_content reinkopiert
            if child.tail:
                text_content.append(child.tail)
    
    # Start der Rekursion mit dem als Argument gepasstem text_element
    recursive_extract(text_element)

    # Kombination aller gesammelten Texte in einen einzelnen String, der durch Leerschläge separiert wird
    return " ".join(text_content)

            

Das nächste Code-Snippet benützt den xml.etree.ElementTree um durch jedes einzelne XML-File zu gehen und die relevanten Informationen zu speichern. Konkret sind dies:

    - Haupttitel
    - Untertitel
    - Volumetitel
    - Hauptklasse (Genre)
    - Subklasse (Subgenre)
    - Autor (Nachname, Vorname)
    - Publikationsdatum
    - Sprache
    - Text

Diese Daten werden extrahiert und in der Form eines Dictionarys in eine Liste data[] gesetzt. Diese Liste wird schliesslich in ein pandas DataFrame umgewandelt und in die SQL-Datenbank eingespeist.

Die Extrahierung von Metadaten ist ziemlich einfach. Die findtext()-function kann als input einen XML-Tag nehmen und dann innerhalb dieses Tags Text finden. Zum Beispiel:

```XML
<author>
    <persName ref="http://d-nb.info/gnd/116201843">
        <surname>Bletzacher</surname>
        <forename>Joseph</forename>
    </persName>
</author>
```

Hier kann der Tag "surname" an findtext() gepasst werden und der Output ist "Bletzacher".

Eine Berücksichtigung muss gemacht werden: der Namensraum. Das Deutsche Textarchiv benutzt das TEI/XML-Basisformat (sie nennen es DTABf). Dieses folgt den Richtlinien der Text Encoding Initiative (TEI). Damit die Extraktion funktioniert (die Elemente und Attribute eindeutig identifiziert werden können), muss ein Namensraum definiert werden. Da das DTA das Basisformat von TEI verwendet, kann dieser Namensraum verwendet werden. Er ist under dieser Webadresse zu finden: http://www.tei-c.org/ns/1.0.

Für mehr Informationen wird hier auf das Deutsche Textarchiv verwiesen: https://www.deutschestextarchiv.de/doku/basisformat/einfuehrung.html

In [ ]:
data = []

# Ein Loop durch den gesamten Ordner mit XML-Dateien
for file_name in os.listdir(path_to_directory):
    
    # Defensive Coding: Nur XML-Dateien sollen berücksichtigt werden
    if file_name.endswith(".xml"):
        # Kreieren eines File-Paths
        file_path = os.path.join(path_to_directory, file_name)

        # Die XML-Datei wird ausgelesen
        tree = ET.parse(file_path)
        root = tree.getroot()
        namespace = {"ns0":"http://www.tei-c.org/ns/1.0"}

        # Extrahierung von Metadaten (ohne Text); 
        # @scheme spezifiziert welchen ClassCode (sub oder main) gemeint ist
        # @type spezifiziert welchen Tag gemeint ist, falls mehrere mit dem gleichen Namen existieren
        main_title = root.findtext('.//ns0:title[@type="main"]', namespaces=namespace)
        sub_title = root.findtext('.//ns0:title[@type="sub"]', namespaces=namespace)
        volume_title = root.findtext('.//ns0:title[@type="volume"]', namespaces=namespace)
        class_main = root.findtext(".//ns0:classCode[@scheme='https://www.deutschestextarchiv.de/doku/klassifikation#dwds1main']", namespaces=namespace)
        class_sub = root.findtext(".//ns0:classCode[@scheme='https://www.deutschestextarchiv.de/doku/klassifikation#dwds1sub']", namespaces=namespace)
        author_surname = root.findtext(".//ns0:surname", namespaces=namespace)
        author_forename = root.findtext(".//ns0:forename", namespaces=namespace)
        author = f"{author_surname}, {author_forename}"
        publication_date_str = root.findtext(".//ns0:sourceDesc/ns0:biblFull/ns0:publicationStmt/ns0:date[@type='publication']", namespaces=namespace)
        language = root.findtext(".//ns0:language", namespaces=namespace)

        # Für den text muss zunächst das ganze Text-Element ausgelesen werden, also alles zwischen <text> und <text />
        text_element = root.find(".//ns0:text", namespaces=namespace)

        # Dann kann dieses Textelement and die rekursive Funktion von oben weitergeleitet werden
        plain_text = extract_text(text_element)

        # Schliesslich werden diese Daten als Dictionary in die Datenliste eingefügt
        data.append({
            "haupttitel" : main_title,
            "untertitel" : sub_title,
            "volumetitel" : volume_title,
            "autor" : author,
            "publikationsjahr" : publication_date_str, # Hier wird der String des Jahres (also z.B. "1600") in einen Integer umgewandelt
            "hauptklasse" : class_main,
            "subklasse" : class_sub,
            "sprache" : language,
            "text" : plain_text
        })


# In einem letzten Schritt wird die Data-Liste (mit Dictionaries) in ein pandas DataFrame umgewandelt, und dann in die SQL-Datenbank eingespeist
df = pd.DataFrame(data)

df.to_sql("my_data", con, index=True, if_exists="replace")

Vorhin wurde die SQL-Datenbank mit einem Index kreiert (index=True). Das Keyword "index" ist aber auch ein SQL-Command, weswegen in diesem Code-Snippet der Name geändert wird:

In [ ]:
cur.execute("ALTER TABLE my_data RENAME COLUMN 'index' to 'text_index';")

Das Publikationsjahr ist im Moment noch ein String, mit dem nächsten Code-Snippet wird dieser aber in ein Integer umgewandelt (für Berechnungen bei der Analyse für das Sampling).

Zunächst wird eine neue Spalte erstellt, die das Publikationsjahr als Integer hat:

In [4]:
import sqlite3

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

cur.execute("ALTER TABLE my_data ADD COLUMN publikationsjahr_int INTEGER")

cur.execute("""
    UPDATE my_data
    SET publikationsjahr_int = CAST(publikationsjahr AS INTEGER)
""")
con.commit()

Die Spalte Publikationsjahr mit den Strings wird gelöscht.

In [ ]:
cur.execute("""
            ALTER TABLE my_data
            DROP COLUMN publikationsjahr
            """)

Die neue Integer-Spalte wird umbenannt in Publikationsjahr

In [ ]:
cur.execute("""
    ALTER TABLE my_data
    RENAME COLUMN 'publikationsjahr_int' to 'publikationsjahr'
""")

Überprüfen des Datentyps:

In [ ]:
print(type(cur.execute("SELECT publikationsjahr FROM my_data WHERE text_index=0").fetchone()[0]))

In [ ]:
print(cur.execute("SELECT publikationsjahr FROM my_data WHERE text_index=0").fetchone()[0])

## 6. Tests ##
In diesem Abschnitt wird geschaut, ob die Datenbank erfolgreich aufgesetzt wurde. Dabei wird ein zufällig gewählter text-index genommen und die Informationen gezeigt.

Zufälliger text_index:

In [ ]:
import random

rand_index = random.randint(0, 4435)
print(rand_index)

Abfrage nach Haupttitel:

In [ ]:
cur.execute("SELECT haupttitel FROM my_data WHERE text_index=?", (rand_index,))
print(cur.fetchone()[0])

Abfrage nach Untertitel:

In [ ]:
cur.execute("SELECT untertitel FROM my_data WHERE text_index=?", (rand_index,))
print(cur.fetchone()[0])

Abfrage nach Volumetitel:

In [ ]:
cur.execute("SELECT volumetitel FROM my_data WHERE text_index=?", (rand_index,))
print(cur.fetchone()[0])

Abfrage nach Autor:

In [ ]:
cur.execute("SELECT autor FROM my_data WHERE text_index=?", (rand_index,))
print(cur.fetchone()[0])

Abfrage nach Publikationsjahr:

In [ ]:
cur.execute("SELECT publikationsjahr FROM my_data WHERE text_index=?", (rand_index,))
print(cur.fetchone()[0])

Abfrage nach Hauptklasse:

In [ ]:
cur.execute("SELECT hauptklasse FROM my_data WHERE text_index=?", (rand_index,))
print(cur.fetchone()[0])

Abfrage nach Subklasse:

In [ ]:
cur.execute("SELECT subklasse FROM my_data WHERE text_index=?", (rand_index,))
print(cur.fetchone()[0])

Abfrage nach Sprache:


In [ ]:
cur.execute("SELECT sprache FROM my_data WHERE text_index=?", (rand_index,))
print(cur.fetchone()[0])

Abfrage für den Text:

In [ ]:
cur.execute("SELECT text FROM my_data WHERE text_index=?", (rand_index,))
print(cur.fetchone()[0])